In [1]:
#----------------------- Residential Confidence----------------------------
residential_gp_cats = { 'Food_daily',
                        'Education',
                        'Safety',
                        'Religion',
                        'Health_services'}
commercial_gp_cats = { 'Shopping_Retail',
                       'Financial_services',
                       'Tourism_and_Entertainment',
                       'Transport'
                        }

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("olx_houses_apartments.csv",usecols=['Latitude','Longitude'])
house_lat_lon = df.values
df_shops = pd.read_csv("olx_shops_offices.csv",usecols=['Latitude','Longitude'])
office_lat_lon = df_shops.values
station_df = pd.read_csv('locations_with_cluster_assignments_3Clusters.csv',usecols=['station_id','latitude','longitude'])
station_bounds = station_df.values
places_df = pd.read_csv('Google_places_unique_data_Categories_redistributed.csv',usecols=['latitude','longitude','SubCategoryClass'])
places_lat_lon = places_df.values

In [3]:
height = 0.00445
width = 0.00510260779

def getUpperLowerBounds(y,x):# sat_lat, sat_long
    xmin , xmax = x - width , x+width
    ymin, ymax = y-height, y + height
    return xmin,ymin,xmax,ymax


In [4]:
new_file = open('station_residence_confidence_result.csv','w')
new_file.write('station_id,res_confidence\n')

for station in station_bounds:
    box = getUpperLowerBounds(station[1],station[2])
    res_points = 0
    com_points = 0
    res_confidence = 0
    for ad in house_lat_lon: # OLX houses  --> Residential
        if ad[1]>=box[0] and ad[1]<=box[2] and \
           ad[0]>=box[1] and ad[0]<=box[3]:
                res_points += 1
    for ad in places_lat_lon: # Google places --> residential
        if ad[2] in residential_gp_cats:
            if ad[1]>=box[0] and ad[1]<=box[2] and \
               ad[0]>=box[1] and ad[0]<=box[3]:
                res_points += 1
        if ad[2] in commercial_gp_cats: #Google places --> commercial
            if ad[1]>=box[0] and ad[1]<=box[2] and \
               ad[0]>=box[1] and ad[0]<=box[3]:
                com_points += 1
    for ad in office_lat_lon: # OLX offices --> commercial
        # ad[0] : y , ad[1] : x
        #box[xmin,ymin,xmax,ymax]
        if ad[1]>=box[0] and ad[1]<=box[2] and \
           ad[0]>=box[1] and ad[0]<=box[3]:
                com_points += 1
    if res_points==0 and com_points==0:
        res_confidence = 0
    else:
        res_confidence = res_points / ( res_points + com_points )         
    next_line = ",".join(np.array([str(station[0]),str(res_confidence)]))
    new_file.write(next_line)
    new_file.write('\n')
    
new_file.close()